In [1]:
from flightdata import Flight
from flightanalysis import Section, FlightLine
from flightanalysis.flightline import Box
from geometry import GPSPosition, Coord
import plotly.graph_objs as go
import numpy as np


def ntbkfig():
    fig = go.Figure()
    fig.update_yaxes(
        scaleanchor = "x",
        scaleratio = 1,
    )
    fig.update_layout(width=400, height=400)
    return fig

Logger turned on sitting on the pilot position and only moved directly towards centre before taking off. 

In [2]:
# Read the log and generate a section from the entire flight.
fl_definition_log = Flight.from_csv("../../../data/logs/flightlines/flightline_example.csv")

#create a section default coord (NED)
sec = Section.from_flight(fl_definition_log, FlightLine(Coord.from_nothing(), Coord.from_nothing()))

#create a subset containing only the data where alt < 1m and velocity < 1ms, this gives walking out and walking back quite nicely
ground = sec.data.loc[(sec.pos.z > -1) & (sec.bvel.bvx < 1), :]

sec2 = Section(ground)

fig = ntbkfig()
fig.add_trace(go.Scatter(x=sec2.pos.y, y=sec2.pos.x, text=sec2.data.index))
fig.show()


/home/tom/source/PyFlightCoach/geometry/geometry/points.py:103: RuntimeWarning: invalid value encountered in true_divide
  return Points(self.data / other[:, np.newaxis])


Just for fun, try K means clustering to identify the groups

In [3]:
from sklearn.cluster import KMeans

nc = 2
kmeans = KMeans(
       init="random",
       n_clusters=nc,
       n_init=10,
       max_iter=300,
       random_state=42
   )

kmeans.fit(np.array(sec2.data.index).reshape(-1, 1) )

fig = ntbkfig()
secs = []
for clust in range(nc):
    secs.append(Section(sec2.data[kmeans.labels_==clust]))
    fig.add_trace(go.Scatter(x=secs[-1].pos.y, y=secs[-1].pos.x, text=secs[-1].data.index, name="cluster " + str(clust)))
fig.show()

firstsec = secs[0] if secs[0].data.index[0] < secs[1].data.index[0] else secs[1]


Now fit a line to the first cluster

In [4]:
fit = np.polyfit(x=firstsec.x, y=firstsec.y,deg=1)

fig = ntbkfig()
fig.add_trace(go.Scatter(y=firstsec.pos.x, x=firstsec.pos.y, text=firstsec.data.index, name="data"))
fig.add_trace(go.Scatter(y=firstsec.pos.x, x=fit[0] * firstsec.pos.x + fit[1], text=firstsec.data.index, name="fitted"))
fig.show()

In [60]:
fl_definition_log = Flight.from_csv(
    "../../../data/logs/flightlines/flightline_example.csv")

pilot = fl_definition_log.subset(0, 27).data.mean()

box = Box(
    "2point",
    GPSPosition(pilot.global_position_latitude,
                pilot.global_position_longitude),
     (np.arctan(fit[0]) + np.pi )
     )

flightline = FlightLine.from_box(box)


In [64]:
sec = Section.from_flight(fl_definition_log, flightline)

fig = go.Figure()
fig.add_trace(go.Scatter(x=sec.pos.x, y=sec.pos.y, text=sec.data.index))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_layout(width=800, height=800)
fig.show()